In [40]:
import numpy as np
import cvxpy as cp
import tqdm as tqdm
import matplotlib.pyplot as plt


In [33]:
u_f = 1
L_f = 10

u_phi = 1
L_phi = 1

M_f = np.zeros((3,3))
M_f[1,1]= -(u_f*L_f)/(u_f+L_f)
M_f[1,2]= 1/2
M_f[2,1]= 1/2
M_f[2,2]= -(1)/(u_f+L_f)


M_phi = np.zeros((3,3))
M_phi[1,1]= -(u_phi*L_phi)/(u_phi+L_phi)
M_phi[1,0]= 1/2
M_phi[0,1]= 1/2
M_phi[0,0]= -(1)/(u_phi+L_phi)

eta = 10
rho = 0.9
M_0 = np.zeros((3,3))
M_0[0,0]= (1-rho)/(2*u_f)
M_0[1,2]= -1/2*eta
M_0[2,1]= -1/2*eta
M_0[2,2]= (eta*eta)/(2*u_f)

print(M_0)
print(M_f)
print(M_phi)

[[ 5.e-02  0.e+00  0.e+00]
 [ 0.e+00  0.e+00 -5.e+00]
 [ 0.e+00 -5.e+00  5.e+01]]
[[ 0.          0.          0.        ]
 [ 0.         -0.90909091  0.5       ]
 [ 0.          0.5        -0.09090909]]
[[-0.5  0.5  0. ]
 [ 0.5 -0.5  0. ]
 [ 0.   0.   0. ]]


In [34]:
sigma_f = cp.Variable()
sigma_phi = cp.Variable()
constraints = [sigma_f >= 0]
constraints += [sigma_phi >= 0]
constraints += [(M_0 + sigma_f*M_f + sigma_phi*M_phi) << 0]

prob=cp.Problem(cp.Minimize(0),
                  constraints)
prob.solve()


print("The optimal value is", prob.value)
print("A solution is")
print(sigma_f.value, sigma_phi.value)

print(prob.value == 0)

The optimal value is inf
A solution is
None None
False


In [53]:
M_0_cp = cp.Parameter((3,3))
M_0_cp.value = M_0
sigma_f = cp.Variable()
sigma_phi = cp.Variable()
constraints = [sigma_f >= 0]
constraints += [sigma_phi >= 0]
constraints += [(M_0_cp + sigma_f*M_f + sigma_phi*M_phi) << 0]
prob=cp.Problem(cp.Minimize(0), constraints)

resolution =1000
solvable =  np.zeros((resolution,resolution))
etas = np.linspace(0,1,resolution,endpoint=False)
rhos = np.linspace(0,1,resolution,endpoint=False)
for i, eta in tqdm.tqdm(enumerate(etas)):
    for j, rho in enumerate(rhos):
        M_0 = np.zeros((3,3))
        M_0[0,0]= (1-rho)/(2*u_f)
        M_0[1,2]= -1/2*eta
        M_0[2,1]= -1/2*eta
        M_0[2,2]= (eta*eta)/(2*u_f)
        
        M_0_cp.value = M_0

        prob.solve()
#         print("The optimal value is", prob.value)
        if(prob.value == 0):
            solvable[resolution-j,i] = 1
#             print("this is solvable", eta, rho)
#         else:
#             solvable[i,j] = 0


1000it [3:57:53, 14.27s/it]


In [ ]:
print(solvable)
plt.imshow(solvable)
plt.show()